In [8]:
# import torch
# import gc
# import logging

# # del session.logger
# # del session.epoch_logger
# # del session
# # gc.collect()
# # torch.cuda.empty_cache()

# import multiprocessing
# from train.training_session_v2 import TrainingSessionV2, load_training_session
# from config import TrainingConfigV2
# from config import SimpleConvConfig

# data_partition = {
#     "gwilliams2023": {
#         "testing_subjects": [19, 20, 21],
#         "testing_tasks": [0],
#     },
#     "armeini2022": {
#         "testing_subjects": [],
#         "testing_tasks": [0, 1],
#     },
# }

# model_config = SimpleConvConfig(
#     # Str to list of possible conditions
#     mel_normalization=False,
#     conditions={
#         "study": [],
#         "subject": [],
#     },
#     # Channels
#     in_channels=208,
#     out_channels=80,
#     hidden_dim=256,
#     dropout=0.2,
#     initial_batch_norm=True,
#     # # Sensor layout settings
#     # layout_dim=2,
#     # layout_proj=True,
#     # layout_scaling="minmax",
#     # # Merger with spatial attn
#     # merger=False,
#     # merger_emb_type=None,
#     # merger_emb_dim=0,
#     # merger_channels=0,
#     # merger_dropout=0.0,  # Float
#     # merger_conditional=None,
#     # Inital
#     initial_linear=256,
#     initial_depth=1,
#     # Conditional layers
#     conditional_layers=False,
#     conditional_layers_dim=None,  # input or hidden_dim
#     # Conv layer overall structure
#     depth=4,
#     kernel_size=3,
#     growth=1.0,
#     dilation_growth=2,
#     dilation_period=5,
#     glu=1,
#     conv_dropout=0.2,
#     dropout_input=0.1,
#     batch_norm=True,
#     half=True,
#     cnn_pos_encoding=False,
#     # Quantizer
#     quantizer=False,
#     num_codebooks=0,
#     codebook_size=0,
#     quantizer_commitment=0,
#     quantizer_temp_init=0,
#     quantizer_temp_min=0,
#     quantizer_temp_decay=0,
#     # Transformers Encoders
#     transformer_input="continuous",
#     transformer_encoder_emb="sinusoidal",
#     transformer_encoder_layers=4,
#     transformer_encoder_heads=4,
#     # Conformer encoder variant
#     rnn_type="conformer",
#     depthwise_conv_kernel_size=15,
#     use_group_norm=False,
#     convolution_first=False,
#     # Transformer Decoders
#     transformer_decoder_emb=None,
#     transformer_decoder_layers=0,
#     transformer_decoder_heads=0,
#     transformer_decoder_dim=0,
# )

# config = TrainingConfigV2(
#     brain_encoder_config=model_config,
#     data_partition=data_partition,
#     # Ada lora settings
#     # Around 100k total batches an epoch for gwilliams
#     use_adalora=True,
#     adalora_init_r=12,
#     adalora_target_r=4,
#     adalora_tinit=(654 * 3),  # 5% total steps
#     adalora_tfinal=(654 * 8),  # 50-80% total steps
#     adalora_deltaT=(654 * 1),  # 1-5% total steps
#     adalora_lora_alpha=32,
#     adalora_lora_dropout=0.1,
#     adalora_total_step=(654 * 50),
#     # Pre-processing parameters
#     # Brain
#     new_freq=200,
#     frequency_bands={"all": (0.5, 80)},
#     max_random_shift=1.0,
#     window_size=4,
#     window_stride=1,
#     brain_clipping=None,
#     baseline_window=0.5,
#     notch_filter=True,
#     scaling="both",
#     delay=0.15,
#     # Audio
#     audio_model="openai/whisper-tiny.en",
#     # Hyperparameters
#     learning_rate=1e-4,
#     weight_decay=1e-4,
#     epochs=50,
#     steps_per_epoch=450, # 654,
#     batch_size=128,
#     random_test_size=10,
#     seed=42,
#     mel_alignment_objectives = {
#         "clip_loss": 0.6,
#         "mse_loss": 0.4
#     },
#     latent_alignment_objectives= {
#         "cosine_similarity": 0.4,
#         "mse_loss": 0.4,
#         "clip_loss": 0.6,
#         "mmd_loss": 0.0
#     },
#     decode_timestamps=True,
# )

# config.brain_encoder_config.mel_normalization = False
# config.learning_rate = 3e-4
# config.batch_size = 256
# config.steps_per_epoch = 654 # 654
# config.epochs = 50

# config.brain_encoder_config.in_channels = 269
# config.decode_timestamps = True

# # Sensor layout settings
# config.brain_encoder_config.layout_dim=3
# config.brain_encoder_config.layout_proj=False
# config.brain_encoder_config.layout_scaling="midpoint"
# # Merger with spatial attn
# config.brain_encoder_config.merger=True
# config.brain_encoder_config.merger_emb_type='mlp'
# config.brain_encoder_config.merger_emb_dim=1024
# config.brain_encoder_config.merger_channels=269
# config.brain_encoder_config.merger_dropout=0.1  # Float
# config.brain_encoder_config.merger_conditional=None

# # config.brain_clipping = 20

# # config.brain_encoder_config.hidden_dim = 1024
# # config.brain_encoder_config.initial_linear = 1024

# session = TrainingSessionV2(
#     config=config,
#     studies={study: "audiotext" for study in data_partition.keys()},
#     data_path="data",
#     save_path="saves/phase3/combining/channel_merger_269_mlp_1024_midpoint_3D",
#     clear_cache=False,
#     cache_name="/home/ubuntu/cache",
#     download_studies=True,
# )


# # session = load_training_session(
# #     save_path="saves/phase3/objectives/baseline_gwilliams_latent_loss_no_latent_alignment/epoch_39",
# #     studies={"gwilliams2023": "audiotext"},
# #     data_path="data",
# #     cache_name="/home/ubuntu/cache",
# # )

# try:
#     session.train(
#         device="cuda",
#         buffer_size=30,
#         num_workers=(multiprocessing.cpu_count() - 2),
#         max_cache_size=800,
#         current_epoch=0,
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

# # try:
# #     session.pre_process_all_recordings(
# #         buffer_size=30, num_workers=multiprocessing.cpu_count() - 10, max_cache_size=800
# #     )
# # except KeyboardInterrupt as e:
# #     print("Exited")

Loading Gwilliams2023 with batch type audiotext
Loading Armeini2022 with batch type audiotext
Data partitioned on studies ['gwilliams2023', 'armeini2022'].
Train: 159, Unseen Task: 51, Unseen Subject: 12, Unseen Both: 4.

RNNEncoder initialized as conformer with 4 layers, 256 d_model, 4 nhead
	Embedding: sinusoidal, params: 6075392
SimpleConv initialized with 13703274 parameters, cond: ['study', 'subject']
Merger True, merger channels 269
ConvBlocks: 4, hidden_dim: 256, params 2626048
Using torch.bfloat16
Found 64 target modules for AdaLora: ['model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.fc1', 'model.encoder.layers.0.fc2', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.out_proj', 'model.encoder.layers.1.fc1', 'model.encode

Training Epoch 1: 100%|██████████| 159/159 [28:24<00:00, 10.72s/it]


Testing done in 6.03m.
Epoch 1 done in 34.43m. 0.22m/recording.


New best epoch 1 with CER 0.9427 and BLEU 0.0027.
Mel Loss: 6.2599, Clip Loss: 9.5156, MSE: 1.3764
Mel accuracy: 0.0047, Top 5: 0.0235, Top 10: 0.0469


Training Epoch 2: 100%|██████████| 159/159 [28:14<00:00, 10.66s/it]


Testing done in 4.63m.
Epoch 2 done in 32.87m. 0.21m/recording.


...

Testing done in 5.13m.
Epoch 31 done in 33.37m. 0.21m/recording.


Training Epoch 32: 100%|██████████| 159/159 [28:21<00:00, 10.70s/it]


Testing done in 5.16m.
Epoch 32 done in 33.52m. 0.21m/recording.


Training Epoch 33: 100%|██████████| 159/159 [27:39<00:00, 10.44s/it]


Testing done in 5.39m.
Epoch 33 done in 33.06m. 0.21m/recording.
Early stopping at epoch 33. Highest metrics at epoch 22.


Training completed. Highest epoch at 22.


Test unseen_subject at epoch 22. Mel Loss: 6.0775, Clip Loss: 9.4874, MSE: 0.9627
Mel accuracy: 0.0244, Top 5: 0.0958, Top 10: 0.1642
BLEU: 0.0066, ROUGE-1: 0.0581, BERT: 0.3597, CER: 0.9432, SELF-BLEU: 0.1648


Test unseen_task at epoch 22. Mel Loss: 6.1020, Clip Loss: 9.5370, MSE: 0.9496
Mel accuracy: 0.0225, Top 5: 0.0954, Top 10: 0.1629
BLEU: 0.0048, ROUGE-1: 0.0417, BERT: 0.3559, CER: 0.9367, SELF-BLEU: 0.1714


Test unseen_both at epoch 22. Mel Loss: 6.0808, Clip Loss: 9.5029, MSE: 0.9476
Mel accuracy: 0.0259, Top 5: 0.0850, Top 10: 0.1466
BLEU: 0.0044, ROUGE-1: 0.0358, BERT: 0.3573, CER: 0.9181, SELF-BLEU: 0.1660


In [ ]:
import torch
import gc
import logging

# del session.logger
# del session.epoch_logger
# del session
# gc.collect()
# torch.cuda.empty_cache()

import multiprocessing
from train.training_session_v2 import TrainingSessionV2, load_training_session
from config import TrainingConfigV2
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    "armeini2022": {
        "testing_subjects": [],
        "testing_tasks": [0, 1],
    },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=256,
    dropout=0.2,
    initial_batch_norm=True,
    # # Sensor layout settings
    # layout_dim=2,
    # layout_proj=True,
    # layout_scaling="minmax",
    # # Merger with spatial attn
    # merger=False,
    # merger_emb_type=None,
    # merger_emb_dim=0,
    # merger_channels=0,
    # merger_dropout=0.0,  # Float
    # merger_conditional=None,
    # Inital
    initial_linear=256,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=4,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.1,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input="continuous",
    transformer_encoder_emb="sinusoidal",
    transformer_encoder_layers=4,
    transformer_encoder_heads=4,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=15,
    use_group_norm=False,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV2(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    use_adalora=True,
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=(654 * 3),  # 5% total steps
    adalora_tfinal=(654 * 8),  # 50-80% total steps
    adalora_deltaT=(654 * 1),  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=(654 * 50),
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=1e-4,
    weight_decay=1e-4,
    epochs=50,
    steps_per_epoch=450, # 654,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives = {
        "clip_loss": 0.6,
        "mse_loss": 0.4
    },
    latent_alignment_objectives= {
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.6,
        "mmd_loss": 0.0
    },
    decode_timestamps=True,
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256
config.steps_per_epoch = 654 # 654
config.epochs = 50

config.brain_encoder_config.in_channels = 269
config.decode_timestamps = True

# Sensor layout settings
config.brain_encoder_config.layout_dim=3
config.brain_encoder_config.layout_proj=False
config.brain_encoder_config.layout_scaling="midpoint"
# Merger with spatial attn
config.brain_encoder_config.merger=True
config.brain_encoder_config.merger_emb_type='mlp'
config.brain_encoder_config.merger_emb_dim=1024
config.brain_encoder_config.merger_channels=269
config.brain_encoder_config.merger_dropout=0.1  # Float
config.brain_encoder_config.merger_conditional=None

# config.brain_clipping = 20

# config.brain_encoder_config.hidden_dim = 1024
# config.brain_encoder_config.initial_linear = 1024

session = TrainingSessionV2(
    config=config,
    studies={study: "audiotext" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase3/combining/channel_merger_269_mlp_1024_midpoint_3D",
    clear_cache=False,
    cache_name="/home/ubuntu/cache",
    download_studies=True,
)


# session = load_training_session(
#     save_path="saves/phase3/objectives/baseline_gwilliams_latent_loss_no_latent_alignment/epoch_39",
#     studies={"gwilliams2023": "audiotext"},
#     data_path="data",
#     cache_name="/home/ubuntu/cache",
# )

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=800,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 10, max_cache_size=800
#     )
# except KeyboardInterrupt as e:
#     print("Exited")